In [1]:
'''
DESIGN_GUIDANCE.IPYNB - get analytical guidance for desinging the punisher
'''
# By Kirill Sechkar

# PACKAGE IMPORTS ------------------------------------------------------------------------------------------------------
import numpy as np
import jax
import jax.numpy as jnp
import functools
from diffrax import diffeqsolve, Dopri5, ODETerm, SaveAt, PIDController, SteadyStateEvent
import pandas as pd
import pickle
from bokeh import plotting as bkplot, models as bkmodels, layouts as bklayouts, palettes as bkpalettes, transform as bktransform
from math import pi
from bokeh import plotting as bkplot, models as bkmodels, layouts as bklayouts, io as bkio
from bokeh.colors import RGB as bkRGB
from contourpy import contour_generator as cgen
import time

# set up jax
from jax.lib import xla_bridge
jax.config.update('jax_platform_name', 'cpu')
jax.config.update("jax_enable_x64", True)
print(xla_bridge.get_backend().platform)

# set up bokeh
bkio.reset_output()
bkio.output_notebook() 
bkplot.output_backend = 'svg'

# OWN CODE IMPORTS -----------------------------------------------------------------------------------------------------
import synthetic_circuits as circuits
from cell_model import *
from get_steady_state import *
from Fig2.design_guidance_tools import *

An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.


cpu


Loading BokehJS ...

In [2]:
 # INITIALISE CELL MODEL, LOAD THE CIRCUIT

# initialise cell model
cellmodel_auxil = CellModelAuxiliary()  # auxiliary tools for simulating the model and plotting simulation outcomes
par = cellmodel_auxil.default_params()  # get default parameter values
init_conds = cellmodel_auxil.default_init_conds(par)  # get default initial conditions

# load synthetic gene circuit
ode_with_circuit, circuit_F_calc, par, init_conds, circuit_genes, circuit_miscs, circuit_name2pos, circuit_styles, _ = cellmodel_auxil.add_circuit(
    circuits.punisher_xtra_initialise,
    circuits.punisher_xtra_ode,
    circuits.punisher_xtra_F_calc,
    par, init_conds)  # load the circuit

In [3]:
# SPECIFY THE CIRCUIT'S DEFAULT PARAMETERS

# BURDENSOME SYNTHETIC GENE
par['c_xtra'] = 1
par['a_xtra'] = 1e5

# PUNISHER
# switch gene conc
par['c_switch'] = 10.0  # gene concentration (nM)
par['a_switch'] = 400.0  # promoter strength (unitless)
par['d_switch']=0.01836
# integrase - expressed from the switch gene's operon, not its own gene => c_int, a_int irrelevant
par['k+_int'] = par['k+_switch']/80.0  # RBS weaker than for the switch gene
par['d_int'] = 0.0#0.01836 # rate of integrase degradation per protease molecule (1/nM/h)
# CAT (antibiotic resistance) gene
init_conds['cat_pb'] = 10.0  # gene concentration (nM) - INITIAL CONDITION< NOT PARAMETER as it can be cut out by the integrase
par['a_cat'] = 500.0  # promoter strength (unitless)
# synthetic protease gene
par['c_prot'] = 10.0  # gene concentration (nM)
par['a_prot'] = 25.0  # promoter strength (unitless)
init_conds['p_prot'] = 1500.0 # if zero at start, the punisher's triggered prematurely

# punisher's transcription regulation function
par['K_switch'] = 300.0  # Half-saturation constant for the self-activating switch gene promoter (nM)
par['eta_switch'] = 2 # Hill coefficient for the self-activating switch gene promoter (unitless)
par['baseline_switch'] = 0.025  # Baseline value of the switch gene's transcription activation function
par['baseline_switch_alt'] = 0
par['p_switch_ac_frac'] = 0.92  # active fraction of protein (i.e. share of molecules bound by the inducer)

# CULTURE MEDIUM
init_conds['s'] = 0.5
par['h_ext'] = 10.5 * (10.0 ** 3)

# # initialise cell model
# cellmodel_auxil = CellModelAuxiliary()  # auxiliary tools for simulating the model and plotting simulation outcomes
# par = cellmodel_auxil.default_params()  # get default parameter values
# init_conds = cellmodel_auxil.default_init_conds(par)  # get default initial conditions
# 
# # load synthetic gene circuit
# ode_with_circuit, circuit_F_calc, par, init_conds, circuit_genes, circuit_miscs, circuit_name2pos, circuit_styles, _ = cellmodel_auxil.add_circuit(
#     circuits.punisher_sep_xtra_initialise,
#     circuits.punisher_sep_xtra_ode,
#     circuits.punisher_sep_xtra_F_calc,
#     par, init_conds)  # load the circuit

# # BURDENSOME SYNTHETIC GENE
# par['c_xtra'] = 1
# par['a_xtra'] = 1e5
# 
# # PUNISHER
# # switch gene conc
# par['c_switch'] = 10.0  # gene concentration (nM)
# par['a_switch'] = 2000.0  # promoter strength (unitless)
# par['d_switch']=0.01836
# # integrase 
# par['c_int'] = 10.0  # gene concentration (nM)
# par['a_int'] = par['a_switch'] * 9/25  # promoter strength (unitless)
# par['k+_int'] = 1.0  # integrase binding rate (1/nM/h)
# par['d_int'] = 0  # rate of integrase degradation per protease molecule (1/nM/h)
# # CAT (antibiotic resistance) gene
# init_conds['cat_pb'] = 10.0  # gene concentration (nM) - INITIAL CONDITION< NOT PARAMETER as it can be cut out by the integrase
# par['a_cat'] = 500.0  # promoter strength (unitless)
# # synthetic protease gene
# par['c_prot'] = 10.0  # gene concentration (nM)
# par['a_prot'] = 50.0  # promoter strength (unitless)
# init_conds['p_prot'] = 1500 # if zero at start, the punisher's triggered prematurely
# 
# # punisher's transcription regulation function
# par['K_switch'] = 700.0  # Half-saturation constant for the self-activating switch gene promoter (nM)
# par['eta_switch'] = 2 # Hill coefficient for the self-activating switch gene promoter (unitless)
# par['baseline_switch'] = 0.025  # Baseline value of the switch gene's transcription activation function
# par['baseline_switch_alt'] = 0
# par['p_switch_ac_frac'] = 0.8  # active fraction of protein (i.e. share of molecules bound by the inducer)
# 
# # get the cellular variables in steady state without burden
# e, F_r, h, xis, Ps = values_for_analytical_sep(par, ode_with_circuit, init_conds,
#                                           circuit_genes, circuit_miscs,
#                                           circuit_name2pos,
#                                           circuit_F_calc)

In [4]:
# get the cellular variables in steady state without burden
e, F_r, h, xis, Ps = values_for_analytical(par, ode_with_circuit, init_conds,
                                          circuit_genes, circuit_miscs,
                                          circuit_name2pos,
                                          circuit_F_calc)
# record the cellular variables
cellvars = {'e': e, 'F_r': F_r, # translation elongation rate and ribosome trnscription regulation
            'h': h, # intacellular chlorampenicol concentration
            # burden values
            'xi_a': xis['a'], 'xi_r': xis['r'], 'xi_other_genes': xis['other'], 'xi_cat': xis['cat'],
            'xi_switch_max': xis['switch (max)'], 'xi_int_max': xis['int (max)'], 'xi_prot': xis['prot'],
            # protein degradation correction factors for the switch protein and the integrase
            'P_switch': Ps['switch'], 'P_int': Ps['int']}

In [5]:
# DEFINE THE RANGES OF SHARE OF SWITCH PROTEINS BOUND BY THE INDUCER AND PROTEASE INDUCTION VALUES

# bound (active) fraction range
ac_frac_range=np.linspace(0,1,80)

# protease induction range
fprot_range=np.linspace(0,1,75)
a_prot_default = par['a_prot'] # get default value of a_prot - F_prot values will be baked into this parameter by multiplying a_prot_default by fprot

In [6]:
# ACTIVE FRACTION VS PROTEASE INDUCTION: FIND SWITCHING THRESHOLDS AND GUARANTEED FOLD CHANGES

# get a mesh grid, then flatten its x and y coordinates into a single linear array
ac_frac_mesh, fprot_mesh = np.meshgrid(ac_frac_range, fprot_range)
ac_frac_mesh_ravel = ac_frac_mesh.ravel()
fprot_mesh_ravel = fprot_mesh.ravel()

# specify vmapping axes
par_vmapping_axes = {}
for key in par.keys():
    if (key == 'a_prot' or key == 'p_switch_ac_frac'):
        par_vmapping_axes[key] = 0
    else:
        par_vmapping_axes[key] = None
cellvars_vmapping_axes = {}
for key in cellvars.keys():
    if(key == 'xi_prot' or key== 'P_switch' or key == 'P_int'):
        cellvars_vmapping_axes[key] = 0
    else:
        cellvars_vmapping_axes[key] = None
        
# make a vmappable parameter dictionary
par_for_existence = par.copy()
par_for_existence['p_switch_ac_frac'] = jnp.array(ac_frac_mesh_ravel)
par_for_existence['a_prot'] = a_prot_default * jnp.array(fprot_mesh_ravel)
# make a vmappable cellular variable dictionary
cellvars_for_existence = cellvars.copy()
cellvars_for_existence['xi_prot'] = (cellvars['xi_prot']/par['a_prot']) * a_prot_default * jnp.array(fprot_mesh_ravel)
cellvars_for_existence['P_switch'] = (cellvars['P_switch']/par['a_prot']) * a_prot_default * jnp.array(fprot_mesh_ravel)
cellvars_for_existence['P_int'] = (cellvars['P_int']/par['a_prot']) * a_prot_default * jnp.array(fprot_mesh_ravel)

# # make the checking function vmappable
vmapped_check_if_threshold_exists = jax.jit(jax.vmap(check_if_threshold_exists,
                                                     in_axes=(par_vmapping_axes, cellvars_vmapping_axes)))

# find for which parameter combinations the switching threshold exists
threshold_exists = np.array(vmapped_check_if_threshold_exists(par_for_existence, cellvars_for_existence))

# from now on, only consider parameter combinations where the threshold bifurcation point exists
indices_where_threshold_exists = []
for i in range(0, len(threshold_exists)):
    if (threshold_exists[i]):
        indices_where_threshold_exists.append(i)
ac_frac_mesh_ravel_exists = ac_frac_mesh_ravel[indices_where_threshold_exists]
fprot_mesh_ravel_exists = fprot_mesh_ravel[indices_where_threshold_exists]

# make a vmappable parameter dictionary
par_for_threshold_gfchanges = par.copy()
par_for_threshold_gfchanges['p_switch_ac_frac'] = jnp.array(ac_frac_mesh_ravel_exists)
par_for_threshold_gfchanges['a_prot'] = a_prot_default * jnp.array(fprot_mesh_ravel_exists)
# make a vmappable cellular variable dictionary
cellvars_for_threshold_gfchanges = cellvars.copy()
cellvars_for_threshold_gfchanges['xi_prot'] = (cellvars['xi_prot']/par['a_prot']) * a_prot_default * jnp.array(fprot_mesh_ravel_exists)
cellvars_for_threshold_gfchanges['P_switch'] = (cellvars['P_switch']/par['a_prot']) * a_prot_default * jnp.array(fprot_mesh_ravel_exists)
cellvars_for_threshold_gfchanges['P_int'] = (cellvars['P_int']/par['a_prot']) * a_prot_default * jnp.array(fprot_mesh_ravel_exists)

# make the threshold and guaranteed fold change retrieval function vmappable
vmapped_threshold_gfchanges = jax.jit(jax.vmap(threshold_gfchanges,
                                               in_axes=(par_vmapping_axes, cellvars_vmapping_axes)))

# find switching thresholds and guaranteed fold changes for the parameter combinations where the switching threshold exists
thresholds_gfchanges = np.array(vmapped_threshold_gfchanges(par_for_threshold_gfchanges, cellvars_for_threshold_gfchanges))

# get the switching thresholds and guaranteed fold changes
xi_thresholds = thresholds_gfchanges[:, 1]
gfchange_intact = thresholds_gfchanges[:, 4]

In [7]:
# ACTIVE FRACTION VS PROTEASE INDUCTION: FIND BURDEN CONTOURS

# fill the points where no threshold exists with INFS
xi_thresholds_for_contour_ravel = np.zeros(threshold_exists.shape)  # initialise
last_index_in_exist_list = 0
for i in range(0, len(xi_thresholds_for_contour_ravel)):
    if (i == indices_where_threshold_exists[last_index_in_exist_list]):
        xi_thresholds_for_contour_ravel[i] = xi_thresholds[last_index_in_exist_list]
        if (last_index_in_exist_list < len(indices_where_threshold_exists) - 1):
            last_index_in_exist_list += 1
    else:
        xi_thresholds_for_contour_ravel[i] = np.inf
xi_thresholds_for_contour = xi_thresholds_for_contour_ravel.reshape(len(fprot_range),
                                                                    len(ac_frac_range)).T

# create a contour generator
threshold_cgen = cgen(x=fprot_range, y=ac_frac_range,
                      z=xi_thresholds_for_contour)

# contours to be found: 1) all synth. genes functional; 2) just the CAT and protease genes functional
xi_native_prot_cat = cellvars['xi_a'] + cellvars['xi_r'] + cellvars['xi_cat'] + cellvars['xi_prot']
xi_with_all_genes = xi_native_prot_cat + cellvars['xi_other_genes']
xi_contours = {'values': [xi_with_all_genes, xi_native_prot_cat],
               'legends': ['Burdensome\ngene\npresent', 'Burdensome gene mutated'],
               'dashes': ['dashed','solid']}
# find burden contour lines
xi_contours['contour lines'] = []
for i in range(0, len(xi_contours['values'])):
    xi_contours['contour lines'].append(threshold_cgen.lines(xi_contours['values'][i]))

In [8]:
# ACTIVE FRACTIONS VS PROTEASE INDUCTION: FIND COMBINATIONS STANDING FOR THE SAME SWITCHING THRESHOLD

ac_fracs_same_threshold = par['p_switch_ac_frac'] / (1 + cellvars['P_switch']) * (1 + (cellvars['P_switch']/par['a_prot']) * a_prot_default * fprot_range)

In [9]:
# ACTIVE FRACTIONS VS PROTEASE INDUCTION: PLOT

# rect widths and heights
rect_widths_along_x_axis = np.zeros(len(fprot_range))
rect_widths_along_x_axis[0] = fprot_range[1] - fprot_range[0]
for i in range(1, len(fprot_range)):
    rect_widths_along_x_axis[i] = ((fprot_range[i] - fprot_range[i - 1]) -
                                   rect_widths_along_x_axis[i - 1] / 2) * 2
rect_heights_along_y_axis = np.zeros(len(ac_frac_range))
rect_heights_along_y_axis[0] = ac_frac_range[1] - ac_frac_range[0]
for i in range(1, len(ac_frac_range)):
    rect_heights_along_y_axis[i] = ((ac_frac_range[i] - ac_frac_range[i - 1]) - rect_heights_along_y_axis[i - 1] / 2) * 2
rect_widths_ravel_exists = np.zeros(fprot_mesh_ravel_exists.shape)
rect_heights_ravel_exists = np.zeros(ac_frac_mesh_ravel_exists.shape)
for i in range(0, len(fprot_mesh_ravel_exists)):
    fprot_where = np.argwhere(
        fprot_range == fprot_mesh_ravel_exists[i])  # locate the baseline value in the protease gene induction range
    rect_widths_ravel_exists[i] = rect_widths_along_x_axis[fprot_where[0][0]]*1.25
    ac_frac_where = np.argwhere(ac_frac_range == ac_frac_mesh_ravel_exists[i])  # locate the eta value in the active fractions range
    rect_heights_ravel_exists[i] = rect_heights_along_y_axis[ac_frac_where[0][0]]*1.25
    
# make a dataframe for the heatmap of guaranteed fold changes
heatmap_df = pd.DataFrame({'F_prot': fprot_mesh_ravel_exists, 'I': ac_frac_mesh_ravel_exists,
                           'gfchange_intact': gfchange_intact,
                           'rect_width': rect_widths_ravel_exists, 'rect_height': rect_heights_ravel_exists})

# plot
gfchange_intact_figure = bkplot.figure(
    frame_width=240,
    frame_height=180,
    x_axis_label="Protease gene induction",
    y_axis_label="Share of switch proteins bound by inducer",
    x_range=(min(fprot_range), max(fprot_range)),
    y_range=(min(ac_frac_range), max(ac_frac_range)),
    tools='pan,box_zoom,reset,save'
)

# plot the heatmap
rects = gfchange_intact_figure.rect(x="F_prot", y="I", source=heatmap_df,
                                 width='rect_width', height='rect_height',
                                 fill_color=bktransform.log_cmap('gfchange_intact',
                                                                    bkpalettes.Plasma256,
                                                                    low=1,
                                                                    high=max(heatmap_df['gfchange_intact'])),
                                 line_width=0,line_alpha=0)
# add colour bar
gfchange_intact_figure.add_layout(rects.construct_color_bar(
    major_label_text_font_size="8pt",
    ticker=bkmodels.FixedTicker(ticks=[1, 10, 100, 1000, 10000]),
    formatter=bkmodels.PrintfTickFormatter(format="%d"),
    label_standoff=6,
    border_line_color=None,
    padding=5
), 'right')

# plot the points where the same switching threshold is reached
gfchange_intact_figure.line(x=fprot_range, y=ac_fracs_same_threshold,
            line_width=2, line_color='white', line_dash='solid')
        
# mark where the point with default parameters lays
gfchange_intact_figure.scatter(marker='x', x=[par['a_prot']/a_prot_default], y=[par['p_switch_ac_frac']],
                               size=10, color='black',line_width=4)

# plot the burden contours
for i in range(0,len(xi_contours['values'])):
    for j in range(0,len(xi_contours['contour lines'][i])):
        gfchange_intact_figure.line(xi_contours['contour lines'][i][j][:, 0], xi_contours['contour lines'][i][j][:, 1],
                    line_dash=xi_contours['dashes'][i],
                    #legend_label=xi_contours['legends'][i],
                    line_width=2, line_color='black')

# add burden contour labels
gfchange_intact_figure.add_layout(bkmodels.Label(x=xi_contours['contour lines'][0][0][-1, 0],
                                 y=xi_contours['contour lines'][0][0][-1, 1],
                                 x_offset=-2, y_offset=24,
                                 text_align='right',
                                 text=xi_contours['legends'][0],
                                 text_font_size='8pt',
                                 text_color='black'))
gfchange_intact_figure.add_layout(bkmodels.Label(x=xi_contours['contour lines'][1][0][-1, 0],
                                 y=xi_contours['contour lines'][1][0][-1, 1],
                                 x_offset=-2, y_offset=-80,
                                 text_align='right',
                                 text=xi_contours['legends'][1],
                                 text_font_size='8pt',
                                 text_color='black'))

# font size
gfchange_intact_figure.xaxis.axis_label_text_font_size = "8pt"
gfchange_intact_figure.xaxis.major_label_text_font_size = "8pt"
gfchange_intact_figure.yaxis.axis_label_text_font_size = "8pt"
gfchange_intact_figure.yaxis.major_label_text_font_size = "8pt"

# show plot
bkplot.show(gfchange_intact_figure)

In [10]:
# GET SWITCHING TIMESCALES FOR THE SAME SWITCHING THRESHOLD AND DIFFERENT PROTEASE INDUCTION VALUES

# set deterministic simulation parameters
savetimestep = 0.01  # save time step
rtol = 1e-6  # relative tolerance for the ODE solver
atol = 1e-6  # absolute tolerance for the ODE solver
tf_nopun = (0,25) # time frame for simulation before the punisher comes online
tf_pun = (25,50) # time frame for simulation after the punisher comes online
tf_afterloss = (50,500) # time frame for simulation after burdensome gene loss

# initialise the array of switching times
fprot_range_times=np.linspace(fprot_range[0],fprot_range[-1],25)
ac_fracs_same_threshold_times=par['p_switch_ac_frac'] / (1 + cellvars['P_switch']) * (1 + (cellvars['P_switch']/par['a_prot']) * a_prot_default * fprot_range_times)
cross_times = np.zeros_like(fprot_range_times)

# initialise system parameters
par_timescales = par.copy()
par_timescales['k_sxf'] = 0.0  # ignore integrase action
cellvars_timescales = cellvars.copy()
# cycle through all parameter combinations
for comb_cntr in range(0, len(fprot_range_times)):
    # set the parameters of the system to the combination considered
    par_timescales['a_prot'] = a_prot_default * fprot_range_times[comb_cntr]
    par_timescales['p_switch_ac_frac'] = ac_fracs_same_threshold_times[comb_cntr]
    cellvars_timescales['xi_prot'] = (cellvars['xi_prot']/par['a_prot']) * par_timescales['a_prot']
    cellvars_timescales['P_switch'] = (cellvars['P_switch']/par['a_prot']) * par_timescales['a_prot']
    cellvars_timescales['P_int'] = (cellvars['P_int']/par['a_prot']) * par_timescales['a_prot']
    
    # get the border between the equilibria's basins of attraction
    border = find_basin_border(par_timescales, cellvars_timescales)
    
    # simulate the loss of the burdensome gene
    # initial simulation to get the steady state without gene expression loss
    p_switch_ac_frac = par_timescales['p_switch_ac_frac']
    par_timescales['p_switch_ac_frac'] = 0.0  # set the burdensome gene expression to present
    sol=ode_sim(par_timescales,    # dictionary with model parameters
                ode_with_circuit,   #  ODE function for the cell with synthetic circuit
                cellmodel_auxil.x0_from_init_conds(init_conds,circuit_genes,circuit_miscs),  # initial condition VECTOR
                len(circuit_genes), len(circuit_miscs), circuit_name2pos, # dictionaries with circuit gene and miscellaneous specie names, species name to vector position decoder
                cellmodel_auxil.synth_gene_params_for_jax(par_timescales,circuit_genes), # synthetic gene parameters for calculating k values
                tf_nopun, jnp.arange(tf_nopun[0], tf_nopun[1], savetimestep), # time frame and time axis for saving the system's state
                rtol, atol)    # relative and absolute tolerances
    ts_nopun=np.array(sol.ts)
    xs_nopun=np.array(sol.ys)
    par_timescales['p_switch_ac_frac'] = p_switch_ac_frac  # set the burdensome gene expression back to present
    sol=ode_sim(par_timescales,    # dictionary with model parameters
                ode_with_circuit,   #  ODE function for the cell with synthetic circuit
                xs_nopun[-1,:],  # initial condition VECTOR
                len(circuit_genes), len(circuit_miscs), circuit_name2pos, # dictionaries with circuit gene and miscellaneous specie names, species name to vector position decoder
                cellmodel_auxil.synth_gene_params_for_jax(par_timescales,circuit_genes), # synthetic gene parameters for calculating k values
                tf_pun, jnp.arange(tf_pun[0], tf_pun[1], savetimestep), # time frame and time axis for saving the system's state
                rtol, atol)    # relative and absolute tolerances
    ts_preloss=np.array(sol.ts)
    xs_preloss=np.array(sol.ys)
        
    # simulating synthetic gene expression loss
    x0_afterloss=sol.ys[-1,:]  # simulation will resume from the last time point
    par_timescales['func_xtra'] = 0.0  # set the burdensome gene expression to zero
    sol=ode_sim(par_timescales,    # dictionary with model parameters
                ode_with_circuit,   #  ODE function for the cell with synthetic circuit
                x0_afterloss,  # initial condition VECTOR
                len(circuit_genes), len(circuit_miscs), circuit_name2pos, # dictionaries with circuit gene and miscellaneous specie names, species name to vector position decoder
                cellmodel_auxil.synth_gene_params_for_jax(par_timescales,circuit_genes), # synthetic gene parameters for calculating k values
                tf_afterloss, jnp.arange(tf_afterloss[0], tf_afterloss[1], savetimestep), # time frame and time axis for saving the system's state
                rtol, atol)    # relative and absolute tolerances
    ts_afterloss = np.array(sol.ts)
    xs_afterloss = np.array(sol.ys)
    par_timescales['func_xtra'] = 1.0 # set the burdensome gene back to present
    
    # get the first time point where p_switch crosses the border
    cross_times[comb_cntr] = ts_afterloss[np.argwhere(xs_afterloss[:,circuit_name2pos['p_switch']] > border)[0][0]]-ts_afterloss[0]

In [11]:
# PLOT THE SWITCHING TIMESCALES

timescales_figure = bkplot.figure(
    frame_width=240,
    frame_height=180,
    x_axis_label="Protease gene induction",
    y_axis_label="Switching timescale (h)",
    x_range=(min(fprot_range_times), max(fprot_range_times)),
    tools='pan,box_zoom,reset,save'
)

# plot the switching timescales
timescales_figure.line(x=fprot_range_times, y=cross_times,
            line_width=2, line_color='black')

# # mark where the point with default parameters
# timescales_figure.scatter(marker='x', x=[par['a_prot']/a_prot_default], y=[par['p_switch_ac_frac']],
#                                size=10, color='black',line_width=4)

# show plot
bkplot.show(timescales_figure)                        

In [18]:
# PLOTS FOR ECC 2024

# SWITCHING THRESHOLD TUNING
gfchange_intact_figure_ecc2024 = bkplot.figure(
    frame_width=240,
    frame_height=180,
    x_axis_label="Protease gene induction",
    y_axis_label="Switch gene induction",
    x_range=(min(fprot_range), max(fprot_range)),
    y_range=(min(ac_frac_range), max(ac_frac_range)),
    tools='pan,box_zoom,reset,save'
)

# plot the heatmap
rects = gfchange_intact_figure_ecc2024.rect(x="F_prot", y="I", source=heatmap_df,
                                 width='rect_width', height='rect_height',
                                 fill_color=bktransform.log_cmap('gfchange_intact',
                                                                    bkpalettes.Plasma256,
                                                                    low=1,
                                                                    high=max(heatmap_df['gfchange_intact'])),
                                 line_width=0,line_alpha=0)
# add colour bar
gfchange_intact_figure_ecc2024.add_layout(rects.construct_color_bar(
    major_label_text_font_size="8pt",
    ticker=bkmodels.FixedTicker(ticks=[1, 10, 100, 1000, 10000]),
    formatter=bkmodels.PrintfTickFormatter(format="%d"),
    label_standoff=6,
    border_line_color=None,
    padding=5
), 'right')

# plot the points where the same switching threshold is reached
gfchange_intact_figure_ecc2024.line(x=fprot_range, y=ac_fracs_same_threshold,
            line_width=2, line_color='white', line_dash='solid')
        
# mark where the point with default parameters lays
gfchange_intact_figure_ecc2024.scatter(marker='x', x=[par['a_prot']/a_prot_default], y=[par['p_switch_ac_frac']],
                               size=10, color='black',line_width=4)

# plot the burden contours
for i in range(0,len(xi_contours['values'])):
    for j in range(0,len(xi_contours['contour lines'][i])):
        gfchange_intact_figure_ecc2024.line(xi_contours['contour lines'][i][j][:, 0], xi_contours['contour lines'][i][j][:, 1],
                    line_dash=xi_contours['dashes'][i],
                    #legend_label=xi_contours['legends'][i],
                    line_width=2, line_color='black')

# add burden contour labels
gfchange_intact_figure_ecc2024.add_layout(bkmodels.Label(x=xi_contours['contour lines'][0][0][0, 0],
                                 y=xi_contours['contour lines'][0][0][0, 0],
                                 x_offset=2, y_offset=48,
                                 text_align='left',
                                 text=xi_contours['legends'][0],
                                 text_font_size='8pt',
                                 text_color='black'))
gfchange_intact_figure_ecc2024.add_layout(bkmodels.Label(x=xi_contours['contour lines'][1][0][0, 0],
                                 y=xi_contours['contour lines'][1][0][0, 1],
                                 x_offset=150, y_offset=0,
                                 text_align='right',
                                 text=xi_contours['legends'][1],
                                 text_font_size='8pt',
                                 text_color='black'))

# mark the fast-switching parameter combination
gfchange_intact_figure_ecc2024.scatter(marker='x', x=fprot_range_times[-1], y=ac_fracs_same_threshold_times[-1],
                               size=10, color=bkRGB(117,200,217),line_width=4)
# mark the slow-switching parameter combination
fprot_slow=0.6
ac_frac_slow=par['p_switch_ac_frac'] / (1 + cellvars['P_switch']) * (1 + (cellvars['P_switch']/par['a_prot']) * a_prot_default * fprot_slow)
gfchange_intact_figure_ecc2024.scatter(marker='x', x=fprot_slow, y=ac_frac_slow,
                               size=10, color='black',line_width=4)

# font size
gfchange_intact_figure_ecc2024.xaxis.axis_label_text_font_size = "8pt"
gfchange_intact_figure_ecc2024.xaxis.major_label_text_font_size = "8pt"
gfchange_intact_figure_ecc2024.yaxis.axis_label_text_font_size = "8pt"
gfchange_intact_figure_ecc2024.yaxis.major_label_text_font_size = "8pt"

# SWITCHING TIMESCALES
timescales_figure_ecc2024 = bkplot.figure(
    frame_width=240,
    frame_height=180,
    x_axis_label="Protease gene induction",
    y_axis_label="Switching time, h",
    x_range=(min(fprot_range_times), max(fprot_range_times)),
    y_range=(0.5,4),
    tools='pan,box_zoom,reset,hover,save'
)

# plot the switching timescales
timescales_figure_ecc2024.line(x=fprot_range_times, y=cross_times,
            line_width=2, line_color=bkRGB(187,51,133))

# mark the fast-switching parameter combination
timescales_figure_ecc2024.scatter(marker='x', x=fprot_range_times[-1], y=cross_times[-1],
                               size=10, color=bkRGB(117,200,217),line_width=4)

# mark the slow-switching parameter combination
cross_time_slow = np.interp(fprot_slow, fprot_range_times, cross_times)
timescales_figure_ecc2024.scatter(marker='x', x=fprot_slow, y=cross_time_slow,
                               size=10, color='black',line_width=4)

# font size
timescales_figure_ecc2024.xaxis.axis_label_text_font_size = "8pt"
timescales_figure_ecc2024.xaxis.major_label_text_font_size = "8pt"
timescales_figure_ecc2024.yaxis.axis_label_text_font_size = "8pt"
timescales_figure_ecc2024.yaxis.major_label_text_font_size = "8pt"


# save plots
gfchange_intact_figure_ecc2024.output_backend = 'svg'
timescales_figure_ecc2024.output_backend = 'svg'
bkplot.output_file("ecc2024_times.html")
bkplot.save(bklayouts.column([gfchange_intact_figure_ecc2024, timescales_figure_ecc2024]),
            title='ECC 2024 Time Figure')


'/mnt/c/Users/ersat/CODE/punisher/Fig3/ecc2024_times.html'